# Temat testów:

### Czy istnieje istotna statystycznie różnica w poprawności odpowiedzi w zależności od czasu odpowiedzi? (grupy: wszystkie grafiki, grafiki A, grafiki B, grafiki 1,2,3)

In [1]:
import statsmodels.api as sm
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
df_corr = pd.read_csv('/Users/martasolarz/Studies/Thesis/data/correctness.csv',
    usecols=['ID', '1a_corr', '1b_corr', '2a_corr', '2b_corr', '3a_corr', '3b_corr']
)

df_corr = df_corr[df_corr.ID != 6] # odrzucony uczestnik ze względu na metryki eye trackingowe

df_time =  pd.read_csv('/Users/martasolarz/Studies/Thesis/data/metrics/times.csv',
    usecols=['ID', 'Duration_1a', 'Duration_1b', 'Duration_2a', 'Duration_2b', 'Duration_3a', 'Duration_3b']
)

df_merge = df_corr.merge(df_time, on='ID')
df_merge.head()

,ID,1a_corr,1b_corr,2a_corr,2b_corr,3a_corr,3b_corr,Duration_1a,Duration_1b,Duration_2a,Duration_2b,Duration_3a,Duration_3b
0,1,1,1,0,0,1,1,28015,15516,54481,58697,56314,63897
1,2,1,1,1,1,0,1,66230,88463,42248,85329,47948,129427
2,3,1,1,1,1,0,0,68830,83963,45165,32732,8000,61197
3,4,1,1,1,1,1,0,46048,78446,12449,37715,60980,101795
4,5,1,1,1,1,1,1,67980,27582,55931,22132,34898,62930


In [3]:
df_merge[['Duration_1a', 'Duration_1b', 'Duration_2a', 'Duration_2b', 'Duration_3a', 'Duration_3b']].describe().T

,count,mean,std,min,25%,50%,75%,max
Duration_1a,40.0,52403.675,18085.029493,28015.0,39714.50,52705.5,62955.5,117478.0
Duration_1b,40.0,50364.550,24844.453825,15516.0,30481.50,44781.0,66255.0,112511.0
Duration_2a,40.0,34225.000,23195.850356,8416.0,15745.00,27232.0,45294.0,98295.0
Duration_2b,40.0,55594.400,28714.159003,12066.0,33344.00,53205.5,71830.0,121461.0
Duration_3a,40.0,42925.775,22404.602238,8000.0,22594.75,39856.0,57476.5,100562.0
Duration_3b,40.0,80122.275,35611.747787,18282.0,56526.50,73571.0,105428.0,144209.0


Funkcje:

In [4]:
def prepate_set(list_of_cols_czas, list_of_cols_popr):
    new_row_czas = list_of_cols_czas[0]
    for i in list_of_cols_czas[1:]:
        new_row_czas = pd.concat([new_row_czas, i], ignore_index=True)

    new_row_popr = list_of_cols_popr[0]
    for i in list_of_cols_popr[1:]:
        new_row_popr = pd.concat([new_row_popr, i], ignore_index=True)

    df = pd.DataFrame({'poprawnosc': new_row_popr, 'czas': new_row_czas})
    df['intercept'] = 1 # specjalna kolumna dla regresji logistycznej
    
    return df

In [16]:
def conduct_logistic_regression(df):
    X = df[['czas']] # zmienne niezależne
    y = df['poprawnosc'] # zmienna zależna

    # Tworzenie modelu regresji logistycznej
    model = sm.Logit(y, X)

    # Dopasowanie modelu
    result = model.fit()
    
    print(result.summary())
    
    print(result.pvalues)

a) wszystkie grafiki

In [13]:
popr = []
for col in df_merge.columns.tolist():
    if col.endswith('corr'):
        popr.append(df_merge[col])

czas = []
for col in df_merge.columns.tolist():
    if col.startswith('Duration'):
        czas.append(df_merge[col])
        
df = prepate_set(czas, popr)
df.tail()

In [9]:
df['czas'].describe()

count       240.000000
mean      52605.945833
std       29437.719595
min        8000.000000
25%       30969.000000
50%       47589.000000
75%       68163.500000
max      144209.000000
Name: czas, dtype: float64

In [14]:
conduct_logistic_regression(df)

,poprawnosc,czas,intercept
235,1,55214,1
236,0,113828,1
237,0,93112,1
238,0,35898,1
239,0,119944,1


count       240.000000
mean      52605.945833
std       29437.719595
min        8000.000000
25%       30969.000000
50%       47589.000000
75%       68163.500000
max      144209.000000
Name: czas, dtype: float64